## Tutorial on collision avoidance with optimization fabrics

---

This tutorial will walk you through a first trajectory generator with optimization fabrics.\
Let's start by importing the necessary components for fabrics.

In [ ]:
import casadi as ca
import numpy as np
from fabrics.diffGeometry.diffMap import DifferentialMap
from fabrics.diffGeometry.energy import Lagrangian
from fabrics.diffGeometry.geometry import Geometry
from fabrics.helpers.variables import Variables
from fabrics.diffGeometry.energized_geometry import WeightedGeometry
from fabrics.diffGeometry.speedControl import Damper
from fabrics.components.energies.execution_energies import ExecutionLagrangian

### Simulation

For visualization, we also want to run a simulation of our first planner, using: https://github.com/maxspahn/gym_envs_planar.

In [ ]:
import gym
import planarenvs.point_robot
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

def run_environment(
        weighted_geometry,
        render=False,
        dt = 0.001,
        T= 20,
        obstacles = [],
        init_vel = [1.0, 0.1],
        init_pos = [1.0, 1.0],
    ):
    
    # Using openai gym interface for the simulation
    env = gym.make("point-robot-acc-v0", render=render, dt=dt)
    n_steps = int(T/dt)
    
    positions = []
    ob = env.reset(pos=np.array(init_pos), vel=np.array(init_vel))
    for obstacle in obstacles:
        env.add_obstacle(obstacle)
    for i in range(n_steps):
        x = ob['joint_state']['position']
        xdot = ob['joint_state']['velocity']
        positions.append(x)
                
        # Simple logging to monitor the proces
        if i % (1/dt) == 0:
            print(f"Current time : {i * dt}/{T}")            
        
        # Use a weighted geometry to compute the action
        _, _, xddot, alpha = weighted_geometry.evaluate(q=x,qdot=xdot)
        action = xddot - alpha * xdot

        # Break if something goes horribly wrong
        if np.linalg.norm(action) > 1e5:
            print(f"Breaking due to action magnitude at step {dt*i}, {x}")
            break
        if np.any(np.isnan(action)):
            print(f"Breaking due to isnan at step {dt*i}, {x}")
            break
        if np.linalg.norm(xdot) > 1.2 * np.linalg.norm(np.array(init_vel)):
            print(f"Breaking due to velocity magnitude at step {dt*i}, {x}, {xdot}")
            break

        ob, _, _, _ = env.step(action)
    env.close()
    return np.array(positions)

### Visualizations

We use matplotlib for visualizations of the trajectories.

In [ ]:
import matplotlib.pyplot as plt

def plot_trajectory(positions, goal=None, obstacles = []):
    fig, axs = plt.subplots(1, 1, figsize=(7, 7))
    
    for obstacle in obstacles:
        axs.add_patch(plt.Circle(obstacle.position(), radius=obstacle.radius(), color='#D80032', alpha=0.9, label='Obstacle'))
    
    if goal:
        axs.add_patch(plt.Circle(goal.position(), radius=0.25, color='#00A878', label='Goal position'))
    
    # Initial position
    axs.add_patch(plt.Circle([positions[0,0], positions[0,1]], radius=0.25, color='#F4D35E', label='Initial position'))
    
    # Trajectory
    axs.plot(positions[:,0], positions[:,1], color='#27474E', alpha=0.9, linewidth=3, label='Trajectory')
    
    # Arrows on trajectory
    #m = int(len(positions)/200)
    #arrow_width = 0.05; arrow_head_width = 3*arrow_width; arrow_length = 1.5*arrow_head_width
    #for n in (n+sum(range(n)) for n in (range (1, m, int(m/4)))):
    #    axs.arrow(positions[n, 0], positions[n, 1], positions[n + 1, 0] - positions[n, 0], positions[n + 1, 1] - positions[n, 1],
    #                width=arrow_width, head_width=arrow_head_width, head_length=arrow_length, color='#27474E', alpha=0.95)
    
    
    for item in ([axs.title, axs.xaxis.label, axs.yaxis.label] +
                axs.get_xticklabels() + axs.get_yticklabels()):
        item.set_fontsize(19)
    plt.rc('font', size=19)
    #axs.axis('equal')
    axs.axis('on')
    axs.set_xlabel("x")
    axs.set_ylabel("y")
    axs.legend(loc=2)
    axs.set_xlim([0, 6])
    axs.set_ylim([0, 6])
    return plt

### Base geometry
Now, we can start building the planner.\
We define our configuration space variables $\bf{q} \in \mathcal{Q} = \mathcal{R}^2$ and $\dot{\bf{q}} \in \mathcal{R}^2$.

Then, we define a base geometry using the `Geometry`-class with a zero forcing term $\bf{h}_b = \bf{0}$.\
We equip it with a `Lagrangian` with the energy $\mathcal{L}e_b = \frac{1}{2}\dot{\bf{q}}^T\dot{\bf{q}}$.

We energize the base fabric $(\bf{M}_b, \bf{f}_b)_{\mathcal{Q}}$ by creating an instance of `WeightedGeometry` by providing the Lagrangian and the geometry.

Then, we verify if $\bf{M}_b = \begin{bmatrix}1 & 0 \\ 0 & 1 \end{bmatrix}$ and $\bf{f}_b = \begin{pmatrix}0 \\ 0 \end{pmatrix}$.

> Note that all calculations are done symbolically.

In [ ]:
q = ca.SX.sym('q', 2)
qdot = ca.SX.sym('qdot', 2)
config_variables = Variables(state_variables={'q': q, 'qdot': qdot})
geo_b = Geometry(var=config_variables, h=ca.SX(np.zeros(2)))
lag_b = Lagrangian(0.5 * ca.norm_2(qdot)**2, var=config_variables)
fabric_b = WeightedGeometry(le=lag_b, g=geo_b)
print(f"M={fabric_b.M()}")
print(f"f={fabric_b.f()}")

#### Visualization base geometry
We can simulate and visualize that.\
First, we need to turn the symbolic expression into concrete functions using the `concretize` method.

> This method is available for most components inside the package and is usually followed by an `evaluate` call.\
See run_environment for that call.

What sort of trajectory do you expect when we run it now?

In [ ]:
fabric_b.concretize()
positions_b = run_environment(fabric_b)

In [ ]:
plot_trajectory(positions_b).show()

### Obstacle geometry

Next, we add obstacles to the environment and thus also to the fabric.\
Let us first define some obstacles within the ellipsoid.\
The entries are `[x_obst, y_obst, radius_obst]`.

In [ ]:
from mpscenes.obstacles.sphere_obstacle import SphereObstacle
obstacle_geometries = [
    [3.0, 3.0, 1.0],
]

Based on this information, we create a list of `SphereObstacle` from https://github.com/maxspahn/motion_planning_scenes. \
They can be added to the environment for visualization.

In [ ]:
obstacles = []
for obstacle_geometry in obstacle_geometries:
    circle_dict = {
        'type': 'SphereObstacle',
        'geometry': {
            'position': obstacle_geometry[0:2],
            'radius': obstacle_geometry[2],
        }
    }
    obstacles.append(SphereObstacle(name="CircleObstacle", content_dict = circle_dict))

Similar to the limit geometry, we define a `DifferentialMap`, a `Geometry` and a `Lagrangian` for all obstacles. 

In [ ]:
fabric_o = None
for i, obstacle in enumerate(obstacles):
    x = ca.SX.sym(f"x_obst_{i}", 1)
    xdot = ca.SX.sym(f"xdot_obst_{i}", 1)
    variables = Variables(state_variables = {'x': x, 'xdot': xdot})

    h_obst = -2/(x**2) * xdot**2 * (-0.5 * (ca.sign(xdot) - 1))
    le = 2/(x**2) * xdot**2

    geometry_obstacle = Geometry(var=variables, h=h_obst)
    le_obst = Lagrangian(le, var=variables)
    weighted_geometry = WeightedGeometry(le=le_obst, g=geometry_obstacle)
    phi = ca.norm_2(q - obstacle.position()) / obstacle.radius() - 1
    diff_map = DifferentialMap(phi, config_variables)
    if fabric_o:
        print('adding')
        fabric_o += weighted_geometry.pull(diff_map)
    else:
        print('creating')
        fabric_o = weighted_geometry.pull(diff_map)

In [ ]:
fabric_o_b = fabric_o + fabric_b

l_exec = Lagrangian(0.5 * ca.norm_2(qdot)**2, var=config_variables)
final_geo = WeightedGeometry(g=Geometry(s=fabric_o_b), le=l_exec)
final_geo.concretize()
positions_o_b = run_environment(final_geo, T=20)

In [ ]:
plot_trajectory(positions_o_b, obstacles=obstacles).show()

### Forcing the fabric

So the great power of fabrics is that we can arbitrarily add different geometries (as we did with the limits and the obstacles) and only in the latest step add the forcing potential.

Let's try that out.\
We first define a goal, again from `mpscenes` at https://github.com/maxspahn/motion_planning_scenes.

In [ ]:
from mpscenes.goals.static_sub_goal import StaticSubGoal
goal_dict = {
        "weight": 1.0,
        "is_primary_goal": True,
        "indices": [0, 1],
        "parent_link" : 'origin',
        "child_link" : 'ee_link',
        "desired_position": [5, 4],
        "epsilon" : 0.1,
        "type": "staticSubGoal"
    }
goal = StaticSubGoal(name="goal", content_dict=goal_dict)

psi = ca.norm_2(q-goal.position())**2
h_forcing = 1.5 * ca.gradient(psi, q) 
forcing_geometry = Geometry(var=config_variables, h=h_forcing)

Now, we add the forcing potential to the final geometry that we have defined earlier.\
Therefore, we change the form of the geometry in the canonical acceleration form $(\bf{I},\bf{h})_{\mathcal{Q}}$.

Only then can we add the forcing geometry.

> We let the following simulation run a bit longer because it looks nice and with a smaller time step to avoid numerical issues.

In [ ]:
fabric_tot = fabric_o + fabric_b
composed_geometry = Geometry(s=fabric_tot) + forcing_geometry
l_exec = Lagrangian(0.5 * ca.norm_2(qdot)**2, var=config_variables)
final_geo = WeightedGeometry(g=composed_geometry, le=l_exec)
final_geo.concretize()

To do: add speed control/damping

In [ ]:
positions = run_environment(final_geo, T=5.2, dt=0.005)

In [ ]:
plot_trajectory(positions, goal=goal, obstacles=obstacles).show()

---

## SUCCESS

Nice, you made it!

I hope it helped you a bit to understand how to use optimization fabrics and the package `fabrics`.\
The next step is to understand the `ParameterizedFabricPlanner`.